## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherDatabase/WeatherDatabase.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Desc,Lat,Lng,Hotel Name
0,9,Jamestown,US,71.24,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
1,10,Camacha,PT,74.62,few clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
2,13,Vite,IN,71.22,light rain,17.2833,74.5500,Hotel Dreamland
3,24,Ribeira Grande,PT,75.60,light rain,38.5167,-28.7000,Quinta da Meia Eira
4,28,Hambantota,LK,78.91,broken clouds,6.1241,81.1185,Bungalow 63


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
Mintemp= float(input("Minimum Temperature:"))
print(f"No colder than {Mintemp}")
Maxtemp= float(input("Maximum Temperature:"))
print(f"No warmer than {Maxtemp}")

Minimum Temperature:70
No colder than 70.0
Maximum Temperature:80
No warmer than 80.0


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
tempDF= city_data_df.loc[(city_data_df["Max Temp"] <= Maxtemp) & \
                                       (city_data_df["Max Temp"] >= Mintemp)]

In [5]:
# 4a. Determine if there are any empty rows.
tempDF.isnull().sum()

City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Desc          0
Country       3
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=tempDF.replace('',pd.np.nan).dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country","Max Temp", "Desc", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Desc,Lat,Lng,Hotel Name
1,Adre,TD,79.38,broken clouds,13.4667,22.2000,
9,Jamestown,US,71.24,broken clouds,42.0970,-79.2353,
10,Camacha,PT,74.62,few clouds,33.0833,-16.3333,
13,Vite,IN,71.22,light rain,17.2833,74.5500,
24,Ribeira Grande,PT,75.60,light rain,38.5167,-28.7000,
28,Hambantota,LK,78.91,broken clouds,6.1241,81.1185,
37,Ponta Do Sol,PT,79.00,overcast clouds,32.6667,-17.1000,
43,Falavarjan,IR,76.21,clear sky,32.5553,51.5097,
46,Kolaras,IN,75.72,scattered clouds,25.2333,77.6000,
47,Sao Joao Da Barra,BR,71.19,overcast clouds,-21.6403,-41.0511,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location":"48.8566, 2.3522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    long=row["Lng"]
    params["location"]=f"{lat},{long}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    baseURL='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(baseURL,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found")

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.replace('',pd.np.nan).dropna()

C:\Users\chris\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [10]:
clean_hotel_df

,City,Country,Max Temp,Desc,Lat,Lng,Hotel Name
9,Jamestown,US,71.24,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
10,Camacha,PT,74.62,few clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
13,Vite,IN,71.22,light rain,17.2833,74.5500,Hotel Dreamland
24,Ribeira Grande,PT,75.60,light rain,38.5167,-28.7000,Quinta da Meia Eira
28,Hambantota,LK,78.91,broken clouds,6.1241,81.1185,Bungalow 63
...,...,...,...,...,...,...,...
686,General Pico,AR,75.97,clear sky,-35.6566,-63.7568,Euskadi Hotel
687,Pokhara,NP,70.30,overcast clouds,28.2333,83.9833,OYO 170 Hotel View Point
691,Neunkirchen,DE,76.75,clear sky,49.3500,7.1833,"Holiday Inn Express Neunkirchen, an IHG Hotel"
693,Tenkodogo,BF,75.34,overcast clouds,11.7800,-0.3697,Hôtel Tiéba


In [11]:
# 8a. Create the output File (CSV)
output_data_file='../VacationSearch/WeatherPyVacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Desc}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))